# Домашняя работа 3
## Аршинов Г.Е.

In [1]:
from typing import List
from string import punctuation
from collections import Counter
import numpy as np
from nltk import sent_tokenize
from string import punctuation
from itertools import islice
punctuation += "«»—…“”"
punct = set(punctuation)

In [4]:
import csv

In [2]:
def generate_deletions(word: str) -> List[str]:
    result = []
    for i, char in enumerate(word):
        result.append(''.join(ch for ch_idx, ch in enumerate(word) if ch_idx != i))
    return tuple(result)

In [3]:
generate_deletions("hello")

('ello', 'hllo', 'helo', 'helo', 'hell')

In [6]:
with open("../compling_nlp_hse_course/data/corpus_500.txt", 'r') as f:
    corpus_raw = f.read()
with open('../compling_nlp_hse_course/data/lenta-ru-news.csv', 'r') as archive:
    reader = csv.reader(archive, delimiter=',', quotechar='"')
    for i, line in islice(enumerate(reader), 5000):
        corpus_raw += (line[2].replace('\xa0', ' ') + '\n')

In [7]:
import regex
tokenizer = regex.compile(r"(?:\w+-)?\w+")

In [8]:
from nltk.corpus import stopwords
ru_stopwords = stopwords.words('russian')

In [9]:
corpus = [word.lower() for word in tokenizer.findall(corpus_raw) if word.lower() not in ru_stopwords]

In [10]:
counter = Counter(corpus)

In [11]:
ready_corpus = {}

In [12]:
for word in counter:
    typo_variants = generate_deletions(word)
    for variant in typo_variants:
        if variant not in ready_corpus:
            ready_corpus[variant] = [word]
        else:
            ready_corpus[variant].append(word)

In [45]:
def suggest_correction(word):
    if word in counter:
        return word
    if word in ready_corpus:
        return sorted([(variant, counter[variant]) for variant in ready_corpus[word]], key=lambda x: x[1], reverse=True)[0][0]
    typo_variants = generate_deletions(word)
    result = []
    for variant in typo_variants:
        suggestions = ready_corpus.get(variant)
        if suggestions:
            result.extend(suggestions)
    top_variants = sorted([(variant, counter[variant]) for variant in result], key=lambda x: x[1], reverse=True)
    return next(iter(top_variants), word)

In [39]:
bad = open('../compling_nlp_hse_course/data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('../compling_nlp_hse_course/data/correct_sents.txt', encoding='utf8').read().splitlines()

In [40]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [41]:
import re

In [47]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], suggest_correction(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [48]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.47332667332667333
0.17114351496546432
0.4814517055242908


In [113]:
def suggest_corrections(word):
    if word in counter:
        return [(word, 1)]
    if word in ready_corpus:
        return sorted([(variant, counter[variant]) for variant in ready_corpus[word]], key=lambda x: x[1], reverse=True)
    typo_variants = generate_deletions(word)
    result = []
    for variant in typo_variants:
        suggestions = ready_corpus.get(variant)
        if suggestions:
            result.extend(suggestions)
    return sorted([(variant, counter[variant]) for variant in result], key=lambda x: x[1], reverse=True) or [(word, 0)]

In [49]:
def normalize(text):
    return [word.strip(punctuation) for word in text.lower().split() if word]

def window(collection, size):
    for i in range(len(collection) - size + 1):
        yield tuple(collection[inc] for inc in range(i, i + size))

def pad(collection, left="<SOS>", right="<EOS>"):
    return [left] + collection + [right]

In [71]:
sentences = [pad(pad(normalize(text))) for text in sent_tokenize(corpus_raw)]

In [77]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in sentences:
    unigrams.update(tuple(sentence))
    bigrams.update(tuple(window(sentence, 2)))
    trigrams.update(tuple(window(sentence, 3)))

In [88]:
from scipy.sparse import lil_matrix

In [108]:
matrix = lil_matrix((len(bigrams), len(unigrams)))

id2bigram = list(bigrams)
bigram2id = {bigram:i for i, bigram in enumerate(id2bigram)}
id2word = list(unigrams)
word2id = {word:i for i, word in enumerate(id2word)}

for word_1, word_2, word_3 in trigrams:
    bigram = (word_1, word_2)
    ngram = (word_1, word_2, word_3)
    matrix[bigram2id[bigram], word2id[word_3]] = (trigrams[ngram] / bigrams[bigram])

In [123]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0



for i, true_sentence in enumerate(true):
    bad_sentence = bad[i]
    word_pairs = align_words(true_sentence, bad_sentence)
    
    word_pairs = [("<SOS>", "<SOS>")] + word_pairs
    pred_sent = []
    for pair_idx, pair in islice(enumerate(word_pairs), 1, None):
        word, bad_word = pair
        pred = None
        predicted = suggest_corrections(word)
        
        
        prev_word = word_pairs[word_idx-1][1]
        
        if prev_word not in unigrams:
            pred = predicted[0][0]
            
        
        else:
            
            lm_predicted = []
            for variant, freq in predicted:
                bigram = (prev_word, variant)
                try:
                    prob = matrix[bigram2id[bigram], word2id[variant]]
                except KeyError:
                    prob = 0
                lm_predicted.append((variant, prob))
            if lm_predicted:
                pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
            
        
        if pred is None:
            pred = bad_word
        if pred == word:
            correct += 1
        else:
            mistakes.append((word, bad_word, pred))
        total += 1
            
        if word == bad_word:
            total_correct += 1
            if word !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word == pred:
                mistaken_fixed += 1
    
    if not i % 50:
        print(i)
        print(correct/total)

0
0.7333333333333333
50
0.5797101449275363
100
0.5842293906810035
150
0.5861850443599493
200
0.5868205868205868
250
0.5927286081527727
300
0.5886418269230769
350
0.5906830672598796
400
0.5931338436557643
450
0.5914583333333333
500
0.5932392273402675
550
0.5926680244399185
600
0.5924441017733231
650
0.5897580987409817
700
0.591732283464567
750
0.5917464996315401
800
0.5887850467289719
850
0.5881529348411416
900
0.586024781637213


In [124]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.5863136863136863
0.8119723714504988
0.4474560698288733
